In [1]:
# download the dataset
region = 'us'
!gsutil -m cp -r gs://{region}_audioset/youtube_corpus/v1/features/features.tar.gz /content/

Copying gs://us_audioset/youtube_corpus/v1/features/features.tar.gz...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1/1 files][  2.4 GiB/  2.4 GiB] 100% Done  51.1 MiB/s ETA 00:00:00           
Operation completed over 1 objects/2.4 GiB.                                      


In [2]:
# extract the dataset
!tar -xzf /content/features.tar.gz -C /content/

In [3]:
# import dependencies
import tensorflow as tf
import pandas as pd
import os
import numpy as np

In [4]:
# Function to parse the SequenceExample
def parse_sequence_example(sequence_example_proto):
    # Define the context and feature list structure for parsing
    context_features = {
        "video_id": tf.io.FixedLenFeature([], dtype=tf.string),
        "start_time_seconds": tf.io.FixedLenFeature([], dtype=tf.float32),
        "end_time_seconds": tf.io.FixedLenFeature([], dtype=tf.float32),
        "labels": tf.io.VarLenFeature(tf.int64)
    }
    sequence_features = {
        "audio_embedding": tf.io.FixedLenSequenceFeature([], dtype=tf.string)
    }

    # Parse the input `SequenceExample` proto using the dictionary above
    context_parsed, sequence_parsed = tf.io.parse_single_sequence_example(
        sequence_example_proto,
        context_features=context_features,
        sequence_features=sequence_features
    )

    return context_parsed, sequence_parsed

In [5]:
# Function to get filename list
def get_files():
  # Base directory where the TFRecord files are extracted
  base_dir = '/content/audioset_v1_embeddings/bal_train'

  # List to store the paths of all TFRecord files
  tfrecord_files = []

  file_count = 0
  # Walk through the directory tree
  for subdir, dirs, files in os.walk(base_dir):
      for file in files:
          if file.endswith('.tfrecord'):
              # Construct the full file path and add it to the list
              full_path = os.path.join(subdir, file)
              tfrecord_files.append(full_path)
              file_count += 1

  print(f"Total number of .tfrecord files available: {file_count}")

  return tfrecord_files
  # for path in tfrecord_files:
  #     print(path)

In [6]:
# obtain file list
tfrecord_files = get_files()

Total number of .tfrecord files available: 4070


In [7]:
# Create a dataset from the tfrecord files
dataset = tf.data.TFRecordDataset(tfrecord_files)

In [8]:
# Map the parsing function over the dataset
parsed_dataset = dataset.map(parse_sequence_example)
parsed_dataset

<_MapDataset element_spec=({'labels': SparseTensorSpec(TensorShape([None]), tf.int64), 'end_time_seconds': TensorSpec(shape=(), dtype=tf.float32, name=None), 'start_time_seconds': TensorSpec(shape=(), dtype=tf.float32, name=None), 'video_id': TensorSpec(shape=(), dtype=tf.string, name=None)}, {'audio_embedding': TensorSpec(shape=(None,), dtype=tf.string, name=None)})>

In [9]:
# Function to decode and flatten audio embeddings
def decode_and_flatten_audio_embeddings(audio_embeddings):
    # Convert the audio embeddings from byte strings to uint8 tensors
    audio_embeddings_uint8 = tf.io.decode_raw(audio_embeddings, tf.uint8)
    # Flatten the audio embeddings
    flattened_embeddings = tf.reshape(audio_embeddings_uint8, [-1])
    return flattened_embeddings.numpy()

In [14]:
# Placeholders for records and embeddings
metadata_records = []
embedding_records = []
mean_pooling_records = []

for context_parsed, sequence_parsed in parsed_dataset:
    video_id = context_parsed["video_id"].numpy().decode('utf-8')
    start_time = context_parsed["start_time_seconds"].numpy()
    end_time = context_parsed["end_time_seconds"].numpy()
    labels = tf.sparse.to_dense(context_parsed["labels"]).numpy()
    audio_embeddings = decode_and_flatten_audio_embeddings(sequence_parsed["audio_embedding"])

    # Check if the dimension of the embeddings is exactly 1280
    if audio_embeddings.shape[0] == 1280:
        # Metadata record
        metadata_record = {
            "video_id": video_id,
            "start_time_seconds": start_time,
            "end_time_seconds": end_time,
            "labels": labels.tolist()  # Convert to list for easier DataFrame handling
        }
        metadata_records.append(metadata_record)

        # Embeddings record, prepend video_id to link with metadata
        embedding_record = {"video_id": video_id}
        embedding_record.update({f"feature_{i}": audio_embeddings[i] for i in range(1280)})
        embedding_records.append(embedding_record)

        # Embeddings after mean pooling
        mean_pooling_record = {"video_id": video_id}
        reshaped_embeddings = audio_embeddings.reshape(10, 128)
        mean_pooling = np.mean(reshaped_embeddings, axis=0)
        mean_pooling_record.update({f"feature_{i}": mean_pooling[i] for i in range(128)})
        mean_pooling_records.append(mean_pooling_record)


# Convert to pandas DataFrames
metadata_df = pd.DataFrame(metadata_records)
embeddings_df = pd.DataFrame(embedding_records)
mean_pooling_df = pd.DataFrame(mean_pooling_records)

In [15]:
metadata_df

,video_id,start_time_seconds,end_time_seconds,labels
0,iZcPr3wgJCw,30.0,40.0,[350]
1,iZmn0dUXP9E,40.0,50.0,"[375, 376]"
2,iZ2K_GPtc6g,30.0,40.0,"[501, 503]"
3,iZtF1lUPbEQ,20.0,30.0,"[60, 130, 137]"
4,iZFRfh1Wjt8,420.0,430.0,"[300, 308, 316, 318]"
...,...,...,...,...
21777,2Cmn2lplGfc,30.0,40.0,[338]
21778,2ClQcAd-wJg,30.0,40.0,"[0, 398]"
21779,-Sclnq22t7o,30.0,40.0,[58]
21780,-SD9DkKyOrY,30.0,40.0,"[161, 162, 164, 165, 166, 168, 171, 172]"


In [16]:
embeddings_df

,video_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_1270,feature_1271,feature_1272,feature_1273,feature_1274,feature_1275,feature_1276,feature_1277,feature_1278,feature_1279
0,iZcPr3wgJCw,132,228,153,42,232,96,32,75,131,...,118,255,29,89,0,0,65,185,255,222
1,iZmn0dUXP9E,80,123,108,0,156,196,88,125,144,...,46,255,93,222,0,0,3,185,155,118
2,iZ2K_GPtc6g,170,81,97,86,108,72,153,231,179,...,167,168,154,0,121,113,142,212,50,187
3,iZtF1lUPbEQ,181,113,129,155,154,153,175,9,255,...,0,62,116,150,0,50,93,0,143,0
4,iZFRfh1Wjt8,38,199,5,0,246,64,198,115,51,...,51,150,101,61,56,0,95,255,0,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21777,2Cmn2lplGfc,59,189,0,13,174,153,248,58,127,...,255,255,255,34,117,56,255,255,60,103
21778,2ClQcAd-wJg,50,230,255,233,21,215,255,26,160,...,224,84,68,80,255,46,191,255,0,0
21779,-Sclnq22t7o,23,105,204,22,146,166,70,164,184,...,0,145,0,111,105,133,47,141,13,174
21780,-SD9DkKyOrY,189,103,103,84,124,199,100,217,74,...,0,42,0,255,66,40,255,82,185,224


In [17]:
mean_pooling_df

,video_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127
0,iZcPr3wgJCw,64.9,221.3,85.4,7.4,215.8,123.5,89.7,58.7,102.3,...,124.0,191.2,144.8,65.8,87.1,153.0,112.2,216.3,187.1,173.0
1,iZmn0dUXP9E,93.7,112.2,100.1,20.7,166.8,153.3,93.8,114.8,160.3,...,54.9,181.4,106.5,192.7,50.5,67.4,95.2,123.9,190.5,132.4
2,iZ2K_GPtc6g,170.6,72.8,107.5,83.9,118.0,103.7,150.8,213.0,187.7,...,96.5,204.1,215.9,80.7,77.4,20.7,80.1,190.3,42.6,146.3
3,iZtF1lUPbEQ,170.6,118.4,102.6,128.2,134.6,115.3,83.7,27.1,207.9,...,120.7,27.1,137.5,150.0,90.3,94.5,100.1,136.5,156.4,119.3
4,iZFRfh1Wjt8,73.9,166.3,82.6,17.5,230.2,55.8,187.4,137.7,132.2,...,91.4,149.5,149.7,134.8,124.3,62.7,146.1,166.0,52.4,85.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21777,2Cmn2lplGfc,59.7,189.7,13.3,18.3,179.3,130.6,225.9,48.1,124.6,...,255.0,214.7,253.7,112.6,123.7,90.4,110.9,138.4,58.5,113.1
21778,2ClQcAd-wJg,55.0,147.8,237.5,223.2,126.1,188.9,237.4,76.9,93.5,...,65.6,122.0,178.2,192.2,198.1,175.9,237.6,61.6,182.2,153.4
21779,-Sclnq22t7o,82.0,77.8,171.4,86.6,112.6,113.1,68.8,99.3,167.5,...,147.6,129.8,32.3,148.6,101.8,54.0,85.2,107.8,183.9,112.1
21780,-SD9DkKyOrY,180.4,107.9,81.7,80.9,99.4,222.1,103.5,233.2,55.2,...,42.1,33.7,0.7,233.8,150.8,93.2,255.0,128.7,137.1,230.7


In [18]:
# Save DataFrames to CSV files
metadata_df.to_csv("/content/test_metadata.csv", index=False)
embeddings_df.to_csv("/content/test_embeddings.csv", index=False)
mean_pooling_df.to_csv("/content/test_embeddings_meanpool.csv", index=False)

In [19]:
from google.colab import files

files.download('/content/test_metadata.csv')
files.download('/content/test_embeddings.csv')
files.download('/content/test_embeddings_meanpool.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>